## Converting a QUBO matrix to an Ising Hamiltonian

Hadfield, S. (2021). On the Representation of Boolean and Real Functions as Hamiltonians for Quantum Computing. ACM Transactions on Quantum Computing, 2(4), 1–21. https://doi.org/10.1145/3478519

In [2]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp

In [3]:
# Define the QUBO problem. Here we can change the variables "features", "samples," "alpha" and "seed".
# (Given by OP)
def generate_problem(features, samples, alpha, seed):
    rng = np.random.default_rng(seed=seed)
    n_features = features
    n_samples = samples

    #X = np.ones(n_features) # Feature encoding binary vector 
    V = rng.integers(0,2, size=(n_samples,1)) # Example model target outputs 
    U = rng.random(size=(n_samples,n_features)) # Example model variables (1 feature per column --> (sample x feature) dimensional matrix)

    p_Vj = np.corrcoef(U, V, rowvar=False)[1:,:1] # Correlation with model target outputs
    p_ij = np.corrcoef(U,rowvar=False)            # Correlation between the features

    p_Vj = np.abs(p_Vj)
    p_ij = np.abs(p_ij)

    np.fill_diagonal(p_ij,0)

    s= n_features
    n, m = np.shape(p_Vj)
    matrix_pVj = np.zeros((s,s))
    for i in range(s):
        matrix_pVj[i,i] = p_Vj[i]
    
    # Weighting the independence vs prediction power of variables
    alpha = alpha

    Q = (alpha * p_ij) - (1-alpha)*matrix_pVj

    return Q

Generate a QUBO problem with 5 features, 5 samples, 0.5 alpha and 51 seed:

In [4]:
Q = generate_problem(5, 5, 0.5, 51)

print(Q)

[[-0.14765756  0.14765756  0.35215976  0.05033288  0.06933625]
 [ 0.14765756 -0.35215976  0.05218711  0.42657332  0.34000757]
 [ 0.35215976  0.05218711 -0.05033288  0.0313348   0.2343195 ]
 [ 0.05033288  0.42657332  0.0313348  -0.06933625  0.16140564]
 [ 0.06933625  0.34000757  0.2343195   0.16140564 -0.13338924]]


C:\Users\tpihkako19\AppData\Local\Temp\ipykernel_21300\2802455376.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  matrix_pVj[i,i] = p_Vj[i]


$$
\begin{pmatrix}
-0.14765756 & 0.14765756 & 0.35215976 & 0.05033288 & 0.06933625 \\
0.14765756 & -0.35215976 & 0.05218711 & 0.42657332 & 0.34000757 \\
0.35215976 & 0.05218711 & -0.05033288 & 0.0313348 & 0.2343195 \\
0.05033288 & 0.42657332 & 0.0313348 & -0.06933625 & 0.16140564 \\
0.06933625 & 0.34000757 & 0.2343195 & 0.16140564 & -0.13338924
\end{pmatrix}
$$

Now, this QUBO problem can be written in a minimization formula as (note that the QUBO matrix is symmetric)

$$
\begin{split}
    \min y = &-0.14765756x_0-0.35215976x_1-0.05033288x_2-0.06933625x_3-0.13338924x_4\\
    &+2* \left[ 0.14765756x_0x_1+0.35215976x_0x_2+0.05033288x_0x_3+0.06933625x_0x_4+0.05218711x_1x_2+0.42657332x_1x_3+0.34000757x_1x_4+0.0313348x_2x_3+0.2343195x_2x_4+0.16140564x_3x_4 \right]
\end{split}
$$

To transform this QUBO problem to an Ising Hamiltonian, we need to do the following transformation for every variable $x_i$
$$x_i \rightarrow \frac{1-z_i}{2} .$$

With this transformation, the Ising form for the problem is

$$
\begin{split}
    \min y &\rightarrow -0.14765756(\frac{1-z_0}{2})-0.35215976(\frac{1-z_1}{2})-0.05033288(\frac{1-z_2}{2})-0.06933625(\frac{1-z_3}{2})-0.13338924(\frac{1-z_4}{2})\\
    &\quad +2* \left[ 0.14765756(\frac{1-z_0}{2})(\frac{1-z_1}{2})+0.35215976(\frac{1-z_0}{2})(\frac{1-z_2}{2})+0.05033288(\frac{1-z_0}{2})(\frac{1-z_3}{2})+0.06933625(\frac{1-z_0}{2})(\frac{1-z_4}{2}) \right. \\
    &\quad \left. +0.05218711(\frac{1-z_1}{2})(\frac{1-z_2}{2})+0.42657332(\frac{1-z_1}{2})(\frac{1-z_3}{2})+0.34000757(\frac{1-z_1}{2})(\frac{1-z_4}{2})+0.0313348(\frac{1-z_2}{2})(\frac{1-z_3}{2})+0.2343195(\frac{1-z_2}{2})(\frac{1-z_4}{2})+0.16140564(\frac{1-z_3}{2})(\frac{1-z_4}{2}) \right] \\
    &= -0.14765756(\frac{1-z_0}{2})-0.35215976(\frac{1-z_1}{2})-0.05033288(\frac{1-z_2}{2})-0.06933625(\frac{1-z_3}{2})-0.13338924(\frac{1-z_4}{2})\\
    &\quad + \frac{1}{2} * \left[ 0.14765756(1-z_0)(1-z_1)+0.35215976(1-z_0)(1-z_2)+0.05033288(1-z_0)(1-z_3)+0.06933625(1-z_0)(1-z_4) \right. \\
    &\quad \left. +0.05218711(1-z_1)(1-z_2)+0.42657332(1-z_1)(1-z_3)+0.34000757(1-z_1)(1-z_4)+0.0313348(1-z_2)(1-z_3)+0.2343195(1-z_2)(1-z_4)+0.16140564(1-z_3)(1-z_4) \right] \\
    &= \frac{1}{2}*(-0.14765756-0.35215976-0.05033288-0.06933625-0.13338924)-\frac{1}{2}*(-0.14765756z_0-0.35215976z_1-0.05033288z_2-0.06933625z_3-0.13338924z_4) \\
    &\quad + \frac{1}{2} * ( 0.14765756-0.14765756z_0-0.14765756z_1+0.14765756z_0z_1+0.35215976-0.35215976z_0-0.35215976z_2+0.35215976z_0z_2+0.05033288-0.05033288z_0-0.05033288z_3+0.05033288z_0z_3 \\
    &\quad +0.06933625-0.06933625z_0-0.06933625z_4+0.06933625z_0z_4+0.05218711-0.05218711z_1-0.05218711z_2+0.05218711z_1z_2+0.42657332-0.42657332z_1-0.42657332z_3+0.42657332z_1z_3 \\
    &\quad +0.34000757-0.34000757z_1-0.34000757z_4+0.34000757z_1z_4+0.0313348-0.0313348z_2-0.0313348z_3+0.0313348z_2z_3+0.2343195-0.2343195z_2-0.2343195z_4+0.2343195z_2z_4 \\
    &\quad +0.16140564-0.16140564z_3-0.16140564z_4+0.16140564z_3z_4) \\
    &= -0.23591445z_0-0.3071329z_1-0.30983414z_2-0.3001552z_3-0.33583986z_4+0.07382878z_0z_1+0.17607988z_0z_2+0.02516644z_0z_3+0.03466812z_0z_4 \\
    &\quad +0.02609355z_1z_2+0.21328666z_1z_3+0.17000378z_1z_4+0.0156674z_2z_3+0.11715975z_2z_4+0.08070282z_3z_4 + 0.55621935 .
\end{split}
$$

For QAOA, we do not need to worry about the constant offset $(0.55621935)$. Now, to use this as an Ising Hamiltonian, we replace every $z_i$ with a Pauli-Z operator $\hat{Z}_i$ and the Hamiltonian becomes

$$
\begin{split}
    \hat{H}_C &= -0.23591445\hat{Z}_0-0.3071329\hat{Z}_1-0.30983414\hat{Z}_2-0.3001552\hat{Z}_3-0.33583986\hat{Z}_4+0.07382878\hat{Z}_0\hat{Z}_1+0.17607988\hat{Z}_0\hat{Z}_2+0.02516644\hat{Z}_0\hat{Z}_3+0.03466812\hat{Z}_0\hat{Z}_4 \\
    &\quad +0.02609355\hat{Z}_1\hat{Z}_2+0.21328666\hat{Z}_1\hat{Z}_3+0.17000378\hat{Z}_1\hat{Z}_4+0.0156674\hat{Z}_2\hat{Z}_3+0.11715975\hat{Z}_2\hat{Z}_4+0.08070282\hat{Z}_3\hat{Z}_4 .
\end{split}
$$

We can use this Ising Hamiltonian $\hat{H}_C$ as a cost Hamiltonian in QAOA.

The following Python function does the transformation from a QUBO matrix to an Ising Hamiltonian explained above.

In [5]:
def convert_qubo_to_ising(qubo):
    # Number of qubits
    n = len(qubo)

    # Calculate the offset also, this is not important for the optimization
    offset = 0
    
    operator_list = []

    for i in range(n):
        for j in range(i, n):
            # Initialize the Pauli operator with all I's
            pauli_operator = list("I"*n)
            
            if i == j:
                pauli_operator[i] = "Z"
                ising_value = -(1/2)*np.sum(qubo[i])
            else:
                pauli_operator[i] = "Z"
                pauli_operator[j] = "Z"
                ising_value = (1/2)*qubo[i][j]

            if not ising_value == 0:
                ising_pauli_op = (''.join(pauli_operator), ising_value)
                operator_list.append(ising_pauli_op)

            offset += (1/2)*qubo[i][j]

    operators = SparsePauliOp.from_list(operator_list)

    return operators

In [7]:
ising = convert_qubo_to_ising(Q)
print(ising)

SparsePauliOp(['ZIIII', 'ZZIII', 'ZIZII', 'ZIIZI', 'ZIIIZ', 'IZIII', 'IZZII', 'IZIZI', 'IZIIZ', 'IIZII', 'IIZZI', 'IIZIZ', 'IIIZI', 'IIIZZ', 'IIIIZ'],
              coeffs=[-0.23591445+0.j,  0.07382878+0.j,  0.17607988+0.j,  0.02516644+0.j,
  0.03466812+0.j, -0.3071329 +0.j,  0.02609355+0.j,  0.21328666+0.j,
  0.17000378+0.j, -0.30983414+0.j,  0.0156674 +0.j,  0.11715975+0.j,
 -0.3001552 +0.j,  0.08070282+0.j, -0.33583986+0.j])


We see that the operators have the same coefficients as was calculated above, so this function does the transformation from a QUBO matrix to an Ising Hamiltonian. This SparsePauliOp can be applied to a quantum circuit using Qiskit.